In [1]:

import numpy as np 
import sympy as sp 


In [2]:
from functions2 import *

In [3]:

t, x, y, z, c = sp.symbols("t x y z c")
u1, u2, w1, w2 = sp.symbols("u1, u2, w1, w2")
τ, ξ, γ, ζ = sp.symbols("τ, ξ, γ, ζ")
φ, χ, ψ, ω = sp.symbols("φ, χ, ψ, ω")
lvar = [t, x, y, z]
lwlf = sp.Matrix([u1, u2, w1, w2])
var = sp.Matrix([1, t, x, y, z])

q, p = sp.symbols("q p")

β0 = sp.Function("β_0")(t, x, y, z)
β1 = sp.Function("β_1")(t, x, y, z)
β2 = sp.Function("β_2")(t, x, y, z)
β3 = sp.Function("β_3")(t, x, y, z) 


In [55]:
#Constants in case needed
k = [   sp.symbols(f"{symbol}_{index}")
        for symbol in ("k")
        for index in range(0, 15)
        ]
#Constants used for φ,χ,ψ,ω
f = [   sp.symbols(f"{symbol}_{index}")
        for symbol in ("f")
        for index in range(0, 15)
        ]
n = [   sp.symbols(f"{symbol}_{index}")
        for symbol in ("n")
        for index in range(0, 4)
        ]
#Symbols used to represent the partial derivatives of the vector fields
der = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("δ")
        for index in ("t", "x", "y", "z", "u1", "u2", "w1", "w2")
        ]

In [5]:
def clear_zer_tn(exp):
    s = np.size(exp)
    l = []
    for i in range(0, s):
        if exp[i] != 0:
            l.append(exp[i])
    #l = np.array(l)
    return l

In [6]:
def listify(expr):
    l = []
    for i in expr:
        l.append(i)
    return l

In [7]:
def comb(τ, ξ, γ, ζ):
    return sp.Matrix([τ, ξ, γ, ζ])
def uncomb(eqs):
    return eqs[0], eqs[1], eqs[2], eqs[3]

In [8]:
grid = var*var.T
grid

Matrix([
[1,    t,    x,    y,    z],
[t, t**2,  t*x,  t*y,  t*z],
[x,  t*x, x**2,  x*y,  x*z],
[y,  t*y,  x*y, y**2,  y*z],
[z,  t*z,  x*z,  y*z, z**2]])

In [9]:
gvar = sp.Matrix( clear_zer_tn ( rm_red_t(  flatten(grid))))
gvar.reshape(3, 5)

Matrix([
[   1,   t,    x,   y,    z],
[t**2, t*x,  t*y, t*z, x**2],
[ x*y, x*z, y**2, y*z, z**2]])

In [10]:
lc0 = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("a")
        for index in range(0, 15)
        ]
lc1 = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("b")
        for index in range(0, 15)
        ]
lc2 = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("c")
        for index in range(0, 15)
        ]
lc3 = [
        sp.symbols(f"{symbol}_{index}")
        for symbol in ("d")
        for index in range(0, 15)
        ]
lc = lc0+lc1+lc2+lc3

In [11]:
#Generate the Ansatz of 2nd order polynomial for our functions
τ = sp.Matrix(lc0).T*gvar
ξ = sp.Matrix(lc1).T*gvar
γ = sp.Matrix(lc2).T*gvar
ζ = sp.Matrix(lc3).T*gvar

In [12]:
#Set certain combations to 0 we have found not to appear
dicti = {
    lc1[13]:0,
    lc2[11]:0,
    lc3[10]:0,
}

In [13]:
eqs = comb(τ, ξ, γ, ζ).subs(dicti)
eqs

Matrix([
[a_0 + a_1*t + a_10*x*y + a_11*x*z + a_12*y**2 + a_13*y*z + a_14*z**2 + a_2*x + a_3*y + a_4*z + a_5*t**2 + a_6*t*x + a_7*t*y + a_8*t*z + a_9*x**2],
[           b_0 + b_1*t + b_10*x*y + b_11*x*z + b_12*y**2 + b_14*z**2 + b_2*x + b_3*y + b_4*z + b_5*t**2 + b_6*t*x + b_7*t*y + b_8*t*z + b_9*x**2],
[           c_0 + c_1*t + c_10*x*y + c_12*y**2 + c_13*y*z + c_14*z**2 + c_2*x + c_3*y + c_4*z + c_5*t**2 + c_6*t*x + c_7*t*y + c_8*t*z + c_9*x**2],
[           d_0 + d_1*t + d_11*x*z + d_12*y**2 + d_13*y*z + d_14*z**2 + d_2*x + d_3*y + d_4*z + d_5*t**2 + d_6*t*x + d_7*t*y + d_8*t*z + d_9*x**2]])

In [14]:
#This would be the sorting/mapping function
#deprecated apparently
def sort_poly(expr, sym):
    #sym = lvar
    #obtain the constant part of the expression
    const = expr.copy()
    for i in sym:
        const = const.subs( i , 0 )

    lista = []
    lista.append(const)
    #substract the constant part every time
    for i in sym:
        expr_zer = expr.copy() - const
        for j in sym:
            #substitute all coefficient with 0
            #except the one we are interested in
            if i != j:
                expr_zer = expr_zer.subs( j, 0 )
        #replace the coefficient we are interested in with 1
        #to obtain a clean final expression
        expr_zer = expr_zer.subs( i , 1 )
        lista.append(expr_zer)
    return lista


In [15]:
#Make a dictionary out of paired coefficients or alone standing ones. Works in case of 0,1,2 coefficients together. 
#In this specific case scenario, it's the only amount we expect to encounter anyways
def mkdic(expr, lcons):
    dic = {}
    c = []
    for i in lcons:
        if expr.has(i):
            c.append(i)
    if len(c) == 1:
        dic.update({c[0]:0})
    elif len(c) != 0: 
        #make it for ANY length?
        #print size, to check if its necessary. for more coefficients its easy to implement actually.
        print(np.size(c))
        poly = sp.Poly(expr, c[0], c[1])
        coef = poly.coeffs()
        dic.update({ c[1] : -1 * ( coef[0]/coef[1] ) * c[0] })
    else:
        print("No constant found")
    return dic

In [16]:
#Create the dictionary for an expression 
def comp( expr, lvar, lc, dicti):
    #p = sort_poly(p, lvar)
    #i use the variables t,x,y,z as generators for the polynomial
    p = sp.Poly(expr, *lvar)
    p = p.coeffs()
    #p now is a list with expressions in each entry with 0,1 or 2 symbols added/subtracted. 
    #Thats why mkdic processes each entry of this list separately
    s = np.size(p)
    dic = {}
    for i in range(0, s):
        dic.update( mkdic( p[i], lc) )
    dicti.update(dic)
    return dicti

In [17]:
#Let's make a generalized make_dictionary_function
#Input is a polynomial which is the result of comparing two derivatives of functions
#Something like τ_x = ξ_y , in this simple case it's just 2 constants always because of the order.
#But for later when we have τ_x + ξ_y + φ_w2 = 0 or τ_y + φ_u1 + χ_u2 = 0 we will need something more elaborate.
#obsolete?
def comp_t( expr, lvar, lc, dicti):
    #i use the variables t,x,y,z as generators for the polynomial
    p = sp.Poly(expr, *lvar)
    p = p.coeffs()
    #p now is a list with expressions, the coefficients of t,x,y,z. Those are expressions like f1-f2+k2 etc.
    #Thats why mkdic processes each entry of this list separately
    
    s = np.size(p)
    dic = {}
    
    for i in range(0, s):
        
        c = []
        
        for j in lc:
            if p[i].has(j):
                c.append(j)
        
        if len(c) == 1:
            dic.update({c[0]:0})

        elif len(c) != 0: 
            #make it for ANY length?
            #print size, to check if its necessary. for more coefficients its easy to implement actually.
            print(np.size(c))

            poly = sp.Poly(p[i], *c)
            coef = poly.coeffs()
            dic.update({ c[0] : -1*(np.sum( np.array(coef[1:])*np.array(c[1:]) ) )/coef[0] })
        else:
            print("No constant found")
    
    return dic


In [18]:
def lie_alg(expr1, expr2, der, dic):
    coef = []
    for i in der:
        if expr1.has(i):
            coef.append(i)
    #list with the derivatives
    pol = sp.Poly(expr1, *der)
    pol = pol.coeffs()
    #list with the coefficients of the derivatives
    s = np.size(pol)
    for i in range(0, s):
        pol[i] = pol[i]*( coef[i].subs( coef[i], sp.diff( expr2 , dic[ coef[i] ]  ) ) )
        #this simulates v_1*v_2 with the derivative applied.
    return np.sum(pol)

In [19]:
#Kinda late to implement this but whatever. Didn't use it at all.
def commutator(expr1, expr2, der, dic):
    comm = lieg_alg(expr1, expr2, der, dic) - lieg_alg(expr2, expr1, der, dic)
    return comm

In [20]:
def systemize(expr, lc, lvar):
    sys = sp.Poly(expr, *lvar).coeffs()
    has = []
    for i in lc:
        if expr.has(i) == True:
            has.append(i)
    return sys, has

In [21]:
dic = {}

In [22]:
eqs

Matrix([
[a_0 + a_1*t + a_10*x*y + a_11*x*z + a_12*y**2 + a_13*y*z + a_14*z**2 + a_2*x + a_3*y + a_4*z + a_5*t**2 + a_6*t*x + a_7*t*y + a_8*t*z + a_9*x**2],
[           b_0 + b_1*t + b_10*x*y + b_11*x*z + b_12*y**2 + b_14*z**2 + b_2*x + b_3*y + b_4*z + b_5*t**2 + b_6*t*x + b_7*t*y + b_8*t*z + b_9*x**2],
[           c_0 + c_1*t + c_10*x*y + c_12*y**2 + c_13*y*z + c_14*z**2 + c_2*x + c_3*y + c_4*z + c_5*t**2 + c_6*t*x + c_7*t*y + c_8*t*z + c_9*x**2],
[           d_0 + d_1*t + d_11*x*z + d_12*y**2 + d_13*y*z + d_14*z**2 + d_2*x + d_3*y + d_4*z + d_5*t**2 + d_6*t*x + d_7*t*y + d_8*t*z + d_9*x**2]])

In [23]:
#Implementation of the first set of equations we have from part 1 of the results.
#And coefficient comparison
eq1 = np.array([0, 0, 0, 0, 0, 0])
eq2 = np.array([1, 1, 2, 2, 3, 3])
var1= np.array([x, t, y, t, z, t])
var2= np.array([t, x, t, y, t, z])
expr1 = []
for i in range(0, 6):
    expr = eqs[ eq1[i] ].diff( var1[i] ) - eqs[ eq2[i] ].diff( var2[i])
    sys, has = systemize(expr, lc ,lvar)
    sol = sp.linsolve(sys, *has)
    ndic = dict(zip(has, *sol))
    eqs = eqs.subs(ndic)
    dic.update(ndic)

In [24]:
#Repeat with the next given set
eq1 = np.array([1, 2, 1])
eq2 = np.array([2, 3, 3])
var1= np.array([y, z, z])
var2= np.array([x, y, x])
expr2 = []
for i in range(0, 3):
    expr = eqs[ eq1[i] ].diff( var1[i] ) - eqs[ eq2[i] ].diff( var2[i])
    sys, has = systemize(expr, lc ,lvar)
    sol = sp.linsolve(sys, *has)
    ndic = dict(zip(has, *sol))
    eqs = eqs.subs(ndic)
    dic.update(ndic)

In [25]:
eqs

Matrix([
[a_0 + b_1*x + c_1*y + c_6*x*y + d_1*z + d_11*t*x + d_13*t*y + 2*d_14*t*z + d_4*t + d_6*x*z + d_7*y*z + d_8*t**2/2 + d_8*x**2/2 + d_8*y**2/2 + d_8*z**2/2],
[       b_0 + b_1*t + c_2*y + c_6*t*y + d_11*t**2/2 + d_11*x**2/2 + d_11*y**2/2 + d_11*z**2/2 + d_13*x*y + 2*d_14*x*z + d_2*z + d_4*x + d_6*t*z + d_8*t*x],
[       c_0 + c_1*t + c_2*x + c_6*t*x + d_11*x*y + d_13*t**2/2 + d_13*x**2/2 + d_13*y**2/2 + d_13*z**2/2 + 2*d_14*y*z + d_3*z + d_4*y + d_7*t*z + d_8*t*y],
[                 d_0 + d_1*t + d_11*x*z + d_13*y*z + d_14*t**2 + d_14*x**2 + d_14*y**2 + d_14*z**2 + d_2*x + d_3*y + d_4*z + d_6*t*x + d_7*t*y + d_8*t*z]])

In [26]:
τ, ξ, γ, ζ = uncomb(eqs)

In [27]:
#double derivative equation yield 0 immediately 
#unnecessary

We will now proceed to find out the φ,χ,ψ,ω equations. With a polynomial Ansatz of the first order, which can be proved by derivating the other given equations.

 
 $$\\  τ_{z} - φ_{u1} + χ_{u2}, 
\\  ξ_{y} + φ_{w1} - χ_{w2}, 
\\  - ξ_{z} - φ_{u2} + χ_{u1}, 
\\  - τ_{x} + φ_{u2} + χ_{u1}, 
\\  τ_{y} - φ_{w2} + χ_{w1}, 
\\  - γ_{z} - φ_{w2} - χ_{w1},
\\  ξ_{z} - τ_{x} + 2 φ_{u2}, 
\\  γ_{z} - τ_{y} + 2 φ_{w2}, 
\\  ξ_{z} + τ_{x} - 2 χ_{u1}, 
\\  - γ_{z} - τ_{y} - 2 χ_{w1}, 
\\  φ_{t} + φ_{z} + χ_{x} + ω_{y}, 
\\  - χ_{y} + ψ_{t} + ψ_{z} + ω_{x}, 
\\  φ_{x} + χ_{t} - χ_{z} - ψ_{y}, 
\\  φ_{y} + ψ_{x} + ω_{t} - ω_{z}  


In [28]:
#phi function
p1 = (-eqs[1].diff(z) + eqs[0].diff(x))/2
p2 = (-eqs[2].diff(z) + eqs[0].diff(y))/2
phi = p1*u2 + p2*w2 
phi
#k_0 function of u2, w2!, t,x,y,z also

u2*(b_1/2 + c_6*y/2 + d_11*t/2 - d_11*z/2 - d_14*x - d_2/2 - d_6*t/2 + d_6*z/2 + d_8*x/2) + w2*(c_1/2 + c_6*x/2 + d_13*t/2 - d_13*z/2 - d_14*y - d_3/2 - d_7*t/2 + d_7*z/2 + d_8*y/2)

In [29]:
#chi function
p1 = ( eqs[1].diff(z) + eqs[0].diff(x))/2
p2 = (-eqs[2].diff(z) - eqs[0].diff(y))/2
chi = p1*u1 + p2*w1 
chi               
#k_1  function of u1, w1, t,x,y,z

u1*(b_1/2 + c_6*y/2 + d_11*t/2 + d_11*z/2 + d_14*x + d_2/2 + d_6*t/2 + d_6*z/2 + d_8*x/2) + w1*(-c_1/2 - c_6*x/2 - d_13*t/2 - d_13*z/2 - d_14*y - d_3/2 - d_7*t/2 - d_7*z/2 - d_8*y/2)

Ansatz: $u_1, u_2, w_1, w_2$ are polynomials of the first order

In [30]:
#Ansatz of linearity for w1 and u1
#But are t,x,y,z also linear in the product with them? or of higher order?
#we know that φ,χ,ψ,ω musst satisfy the wave equation as well. 1st order does, but so does 2nd order as well...
phi = phi + (sp.Matrix(f).T*gvar)[0]*w1 
phi = phi + (sp.Matrix(k).T*gvar)[0]*u1 + n[0]*β0
phi

n_0*β_0(t, x, y, z) + u1*(k_0 + k_1*t + k_10*x*y + k_11*x*z + k_12*y**2 + k_13*y*z + k_14*z**2 + k_2*x + k_3*y + k_4*z + k_5*t**2 + k_6*t*x + k_7*t*y + k_8*t*z + k_9*x**2) + u2*(b_1/2 + c_6*y/2 + d_11*t/2 - d_11*z/2 - d_14*x - d_2/2 - d_6*t/2 + d_6*z/2 + d_8*x/2) + w1*(f_0 + f_1*t + f_10*x*y + f_11*x*z + f_12*y**2 + f_13*y*z + f_14*z**2 + f_2*x + f_3*y + f_4*z + f_5*t**2 + f_6*t*x + f_7*t*y + f_8*t*z + f_9*x**2) + w2*(c_1/2 + c_6*x/2 + d_13*t/2 - d_13*z/2 - d_14*y - d_3/2 - d_7*t/2 + d_7*z/2 + d_8*y/2)

In [31]:
phi = sp.collect(phi, [t, x, y, z])
phi

n_0*β_0(t, x, y, z) + u1*(k_0 + k_12*y**2 + k_14*z**2 + k_4*z + k_5*t**2 + k_9*x**2 + t*(k_1 + k_6*x + k_7*y + k_8*z) + x*(k_10*y + k_11*z + k_2) + y*(k_13*z + k_3)) + u2*(b_1/2 + c_6*y/2 - d_2/2 + t*(d_11/2 - d_6/2) + x*(-d_14 + d_8/2) + z*(-d_11/2 + d_6/2)) + w1*(f_0 + f_12*y**2 + f_14*z**2 + f_4*z + f_5*t**2 + f_9*x**2 + t*(f_1 + f_6*x + f_7*y + f_8*z) + x*(f_10*y + f_11*z + f_2) + y*(f_13*z + f_3)) + w2*(c_1/2 + c_6*x/2 - d_3/2 + t*(d_13/2 - d_7/2) + y*(-d_14 + d_8/2) + z*(-d_13/2 + d_7/2))

In [32]:
#psi - w_2 relation ii)
rel1 = eqs[0].diff(z) - sp.Derivative(φ, u1) + sp.Derivative(χ, u2)
rel1

d_1 + 2*d_14*t + d_6*x + d_7*y + d_8*z - Derivative(φ, u1) + Derivative(χ, u2)

In [33]:
rel2 = eqs[1].diff(y) + sp.Derivative(φ, w1) - sp.Derivative(χ, w2)
rel2

c_2 + c_6*t + d_11*y + d_13*x + Derivative(φ, w1) - Derivative(χ, w2)

In [34]:
#find the other chi terms
p1 = -( eqs[0].diff(z) - sp.diff(phi, u1) )*u2
p2 = ( eqs[1].diff(y) + sp.diff(phi, w1) )*w2
chi = chi + p1 + p2 + n[1]*β1
chi = sp.collect(chi, [t, x, y, z])

psi = -sp.diff(phi, w2)*u2 + sp.diff(phi, u2)*w2 - sp.diff(phi, w1)*u1 + sp.diff(phi, u1)*w1
psi = sp.collect(psi, [t, x, y, z]) + n[2]*β2

omg = sp.diff(chi, u2)*w2 - sp.diff(chi, w2)*u2 + sp.diff(chi, u1)*w1 - sp.diff(chi, w1)*u1
omg = sp.collect(omg, [t, x, y, z]) + n[3]*β3

In [35]:
eqs2 = sp.Matrix([phi, chi, psi ,omg])
eqs2

Matrix([
[                                                                  n_0*β_0(t, x, y, z) + u1*(k_0 + k_12*y**2 + k_14*z**2 + k_4*z + k_5*t**2 + k_9*x**2 + t*(k_1 + k_6*x + k_7*y + k_8*z) + x*(k_10*y + k_11*z + k_2) + y*(k_13*z + k_3)) + u2*(b_1/2 + c_6*y/2 - d_2/2 + t*(d_11/2 - d_6/2) + x*(-d_14 + d_8/2) + z*(-d_11/2 + d_6/2)) + w1*(f_0 + f_12*y**2 + f_14*z**2 + f_4*z + f_5*t**2 + f_9*x**2 + t*(f_1 + f_6*x + f_7*y + f_8*z) + x*(f_10*y + f_11*z + f_2) + y*(f_13*z + f_3)) + w2*(c_1/2 + c_6*x/2 - d_3/2 + t*(d_13/2 - d_7/2) + y*(-d_14 + d_8/2) + z*(-d_13/2 + d_7/2))],
[n_1*β_1(t, x, y, z) + u1*(b_1/2 + c_6*y/2 + d_2/2 + t*(d_11/2 + d_6/2) + x*(d_14 + d_8/2) + z*(d_11/2 + d_6/2)) + u2*(-d_1 + k_0 + k_12*y**2 + k_14*z**2 + k_5*t**2 + k_9*x**2 + t*(-2*d_14 + k_1 + k_6*x + k_7*y + k_8*z) + x*(-d_6 + k_10*y + k_11*z + k_2) + y*(-d_7 + k_13*z + k_3) + z*(-d_8 + k_4)) + w1*(-c_1/2 - c_6*x/2 - d_3/2 + t*(-d_13/2 - d_7/2) + y*(-d_14 - d_8/2) + z*(-d_13/2 - d_7/2)) + w2*(c_2 + f_0 + f_12*y**2

In [36]:
fsys = [
(-chi.diff(y) + psi.diff(t) + psi.diff(z) + omg.diff(x)   ).subs({n[0]:0, n[1]:0, n[2]:0, n[3]:0  }),
( phi.diff(x) + chi.diff(t) - chi.diff(z) - psi.diff(y)   ).subs({n[0]:0, n[1]:0, n[2]:0, n[3]:0  }),
( phi.diff(y) + psi.diff(x) + omg.diff(t) - omg.diff(z)   ).subs({n[0]:0, n[1]:0, n[2]:0, n[3]:0  }),
( phi.diff(t) + phi.diff(z) + chi.diff(x) + omg.diff(y)   ).subs({n[0]:0, n[1]:0, n[2]:0, n[3]:0  })
]

In [37]:
fsys = sp.Matrix(fsys)
fsys

Matrix([
[-u1*(f_1 + 2*f_5*t + f_6*x + f_7*y + f_8*z) - u1*(f_11*x + f_13*y + 2*f_14*z + f_4 + f_8*t) + u2*(-d_13/2 + d_7/2) + u2*(d_13/2 - d_7/2) - u2*(d_13 + f_10*y + f_11*z + f_2 + f_6*t + 2*f_9*x) - u2*(-d_7 + k_10*x + 2*k_12*y + k_13*z + k_3 + k_7*t) - w1*(-d_14 - d_8/2) + w1*(d_14 + d_8/2) + w1*(k_1 + 2*k_5*t + k_6*x + k_7*y + k_8*z) + w1*(k_11*x + k_13*y + 2*k_14*z + k_4 + k_8*t) + w2*(-d_11/2 + d_6/2) + w2*(d_11/2 - d_6/2) - w2*(d_11 + f_10*x + 2*f_12*y + f_13*z + f_3 + f_7*t) + w2*(-d_6 + k_10*y + k_11*z + k_2 + k_6*t + 2*k_9*x)],
[                                                                                                u1*(f_10*x + 2*f_12*y + f_13*z + f_3 + f_7*t) + u1*(k_10*y + k_11*z + k_2 + k_6*t + 2*k_9*x) + u2*(-d_14 + d_8/2) - u2*(d_14 - d_8/2) + u2*(-2*d_14 + k_1 + 2*k_5*t + k_6*x + k_7*y + k_8*z) - u2*(-d_8 + k_11*x + k_13*y + 2*k_14*z + k_4 + k_8*t) + w1*(f_10*y + f_11*z + f_2 + f_6*t + 2*f_9*x) - w1*(k_10*x + 2*k_12*y + k_13*z + k_3 + k_7*t) - w2*(f_11*x + f_1

In [38]:
lc_f = []
for i in lc0+lc1+lc2+lc3+f+k:
    if fsys.has(i):
        lc_f.append(i)
np.size(lc_f)

35

In [39]:
dic2 = {}

In [43]:
for sys_exp in fsys:
    for ua in lwlf:
        temp = sys_exp.copy()
        for ub in lwlf:
            if ua != ub:
                temp = temp.subs(ub, 0)
        temp = temp.subs(ua, 1)
        sys, has = systemize(temp, lc_f ,lvar)
        sol = sp.linsolve(sys, *has)
        #try:
        ndic = dict(zip(has, *sol))
        #except:
        #    print("Value error")
        fsys = fsys.subs(ndic)
        dic2.update(ndic)

In [44]:
final_eqs = sp.Matrix([eqs[0], eqs[1], eqs[2], eqs[3], phi, chi, psi, omg]).subs(dic2)
final_eqs

Matrix([
[                                                                                                                                                                                                                                                                                                                                                    a_0 + b_1*x + c_1*y + d_1*z + d_11*t*x + d_13*t*y + 2*d_14*t*z + d_4*t + 2*f_4*x*y + t**2*(-2*d_14 - k_1 - k_4)/2 + x**2*(-2*d_14 - k_1 - k_4)/2 + x*z*(-d_11 - f_3 + k_2) + y**2*(-2*d_14 - k_1 - k_4)/2 + y*z*(d_13 + f_2 + k_3) + z**2*(-2*d_14 - k_1 - k_4)/2],
[                                                                                                                                                                                                                                                                                                                                                                                                     

In [56]:
lc_f = []
for i in lc0+lc1+lc2+lc3+f+k+n:
    if final_eqs.has(i):
        lc_f.append(i)
sl = np.size(lc_f)
sl

37

In [57]:
#Lets generate all the possible vector fields
v_fields = []
for i in range(0, sl):
    arr = sp.Matrix(final_eqs.copy())
    for j in range(0, sl):
        if i != j:
            arr = arr.subs(lc_f[j], 0)
    temp = 0
    #could probably do a np.sum( arr ) and be done, but whatever
    for k in range(0, 8):
        temp = temp + arr[k].subs(lc_f[i], 1)*der[k]
    v_fields.append(temp)
np.size(v_fields)

37

$[v_1, v_2] = v_1 \cdot v_2 - v_2 \cdot v_1 = c \cdot v_j$ or $= 0$

enum = range(0, sl)
sp.Matrix( [enum, v_fields]).T

In [ ]:
lvar_f = lvar + [u1, u2, w1, w2]
dic_der = dict(list(zip(der, lvar_f)))
der

In [ ]:
import time 

In [ ]:
#rewritten the original lie_alg() function to include the case of 0, since this can happen now as well
def lie_alg_t(expr1, expr2, der, dic):
    if expr1 == 0 or expr2 == 0:
        return 0
    else: 
        terms = []
        for i in der:
            if expr1.has(i):
                terms.append(i)
        #list with the derivatives
        pol = sp.Poly(expr1, *der)
        pol = pol.coeffs()
        #list with the coefficients of the derivatives
        s = np.size(pol)
        for i in range(0, s):
            pol[i] = pol[i]*( terms[i].subs( terms[i], sp.diff( expr2 , dic[ terms[i] ]  ) ) )
            #this simulates v_1*v_2 with the derivative applied.
        return np.sum(pol)

Skew symmetry:
$$ [u, w] = - [w, u]$$

In [ ]:
t0 = time.time()

In [ ]:
#SKEW SYMMETRY
skew = (np.zeros(sl**2).reshape(sl, sl)).tolist()
for i in range(0, sl):
    for j in range(0, sl):
        skew[i][j] = lie_alg(v_fields[i], v_fields[j], der, dic_der ) - lie_alg(v_fields[j], v_fields[i], der, dic_der )

for i in range(0, sl):
    for j in range(0, i+1):
        if skew[i][j] != -skew[j][i] :
            print("For i: ", i, "and j: ", j)
            print("Not fulifilled")
            
for i in range(0, sl):
    if skew[i][i] != 0:
        print("You fuckit up")
#PASSED!

In [ ]:
t1 = time.time()

Bilinearity:
$$ [qv + pu, w] = q[v, w] + p[u,w]$$

In [ ]:
#Bilinearity
for i in range(0, sl):
    v = v_fields[i]
    for j in range(0, sl):
        u = v_fields[j]
        for k in range(0, sl):
            w = v_fields[k]

            bil1 = sp.expand( lie_alg_t( sp.expand(q*v+p*u), w, der, dic_der ) - lie_alg_t( w, sp.expand(q*v+p*u), der, dic_der ) )
            bil2 = sp.expand(  q*( lie_alg_t( v, w, der, dic_der ) - lie_alg_t( w, v, der, dic_der )  )  )
            bil3 = sp.expand(  p*( lie_alg_t( u, w, der, dic_der ) - lie_alg_t( w, u, der, dic_der )  )  )
            if sp.expand(bil1 - bil2 - bil3) != 0:
                print("Bilinearity not satisfied")
                print(i, j, k)

In [ ]:
t2 = time.time()

Jacobi identity:
$$[u, [v, w]] + [w, [u, v]] + [v, [w, u]] = 0$$
$$a = [v, w]$$
$$b = [u, v]$$
$$c = [w, u]$$

In [ ]:
#JACOBI IDENTITY

for i in range(0, sl):
    u = v_fields[i]
    for j in range(0, sl):
        v = v_fields[j]
        b  = sp.expand(lie_alg_t(u, v, der, dic_der)) - sp.expand(lie_alg_t(v, u, der, dic_der))
        for k in range(0, sl):
            w = v_fields[k]
            a  = sp.expand(lie_alg_t(v, w, der, dic_der)) - sp.expand(lie_alg_t(w, v, der, dic_der))
            
            c  = sp.expand(lie_alg_t(w, u, der, dic_der)) - sp.expand(lie_alg_t(u, w, der, dic_der))
            e1 = sp.expand(lie_alg_t(u, a ,der, dic_der)) - sp.expand(lie_alg_t(a, u ,der, dic_der))
            e2 = sp.expand(lie_alg_t(w, b ,der, dic_der)) - sp.expand(lie_alg_t(b, w ,der, dic_der))
            e3 = sp.expand(lie_alg_t(v, c ,der, dic_der)) - sp.expand(lie_alg_t(c, v ,der, dic_der))
            if sp.expand(e1 + e2 + e3) !=0:
                print("Jacobi Identity not fuliffiled")
                print(i, j, k)

#PASSED!
#commented because it takes too long to evaluated

In [ ]:
t3 = time.time()

In [ ]:
print("Skew symmetry time:", t1-t0)
print("Bilinearity time:", t2-t1)
print("Jacobi identity time:", t3- t2)